In [ ]:
import blosc2
import tifffile
import os
import re
import math
import numpy as np
from tqdm import tqdm
from utils import load_tifstack

In [ ]:
def number_of_digits(n):
    if n == 0:
        return 1
    return math.floor(math.log10(abs(n))) + 1

In [ ]:
def parse_filename(filename):
    pattern = r'chunk_z_y_x_(\d+)_(\d+)_(\d+)\.b2nd'
    match = re.match(pattern, filename)
    if match:
        number_z, number_y, number_x = map(int, match.groups())
        return number_z, number_y, number_x
    return None, None, None

In [ ]:
scroll = load_tifstack("../Scroll2.volpkg/volumes/20230210143520_grids")
digits = number_of_digits(scroll.shape[0])

In [ ]:
folder_path = "scroll2-denoised"
output_folder = "scroll2-denoised/tiffs"

In [ ]:
files = os.listdir(folder_path)
chunk_files = [f for f in files if f.endswith('.b2nd')]
# Create a list with the parsed numbers for each file
file_info = [(f, *parse_filename(f)) for f in chunk_files]

# Sort the files by number_z, number_y, number_x
file_info.sort(key=lambda x: (x[1], x[2], x[3]))

In [ ]:
current_z = None
chunk_size = 800

In [ ]:
for filename, number_z, number_y, number_x in tqdm(file_info):
    filepath = os.path.join(folder_path, filename)
    if current_z is None:
        current_z = number_z
        big_block = np.empty((chunk_size, scroll.shape[1], scroll.shape[2]), dtype=np.uint16)
        #big_block = np.empty((chunk_size, scroll.shape[1], scroll.shape[2]), dtype=np.uint8)
        print(big_block.shape)
    if number_z == current_z:
        chunk = blosc2.open(filepath, mode="r")
        big_block[:chunk.shape[0],number_y:number_y+chunk.shape[1],number_x:number_x+chunk.shape[2]] = chunk[:,:,:]
    else:
        print(f"Flush {current_z}")
        for z_slice in range(0,big_block.shape[0]):
            if digits == 4:
                outpath = os.path.join(output_folder, f"{current_z+z_slice:04d}.tif")
            elif digits == 5:
                outpath = os.path.join(output_folder, f"{current_z+z_slice:05d}.tif")
            tifffile.imwrite(outpath, big_block[z_slice], dtype=np.uint16)
            #tifffile.imwrite(outpath, big_block[z_slice], dtype=np.uint8)
        current_z = number_z
        big_block = np.empty((chunk_size, scroll.shape[1], scroll.shape[2]), dtype=np.uint16)
        #big_block = np.empty((chunk_size, scroll.shape[1], scroll.shape[2]), dtype=np.uint8)
        chunk = blosc2.open(filepath, mode="r")
        big_block[:chunk.shape[0],number_y:number_y+chunk.shape[1],number_x:number_x+chunk.shape[2]] = chunk[:,:,:]
        
for z_slice in range(0,min(big_block.shape[0],scroll.shape[0]-current_z)):
    if digits == 4:
        outpath = os.path.join(output_folder, f"{current_z+z_slice:04d}.tif")
    elif digits == 5:
        outpath = os.path.join(output_folder, f"{current_z+z_slice:05d}.tif")
    tifffile.imwrite(outpath, big_block[z_slice], dtype=np.uint16)
    #tifffile.imwrite(outpath, big_block[z_slice], dtype=np.uint8)